In [16]:
from book_cover import Book
import os
import requests
from IPython.display import Image
from book_cover import Book
from typing import Optional
import json

In [ ]:
# Example: fetch books and show first result
books = Book.get_books("War and Peace", limit=3)
for book in books:
    print(book)




In [ ]:
Book.get_book_cover_image('9781566190275')

In [3]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"


In [4]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [ ]:
image = Book.find_book_cover("War and Peace")
image


In [ ]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

In [ ]:
def record_unknown_question_about_book(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

In [5]:

def find_books(query)-> Optional[list[Book]]:
    books = Book.get_books(query, limit=3)
    
    if books:
        push(f"Found {len(books)} books for {query}")
        return books
    else:
        push(f"No books found for {query}")
        return None


In [5]:
def get_book_cover_image(isbn) -> Optional[Image]:
   result = Book.get_book_cover_image(isbn) 
   return result


   



In [7]:
res = get_book_cover_image("11111")
print(res)
display(res)



Content is not an image. Content-Type: 
None


None

In [ ]:
res = find_books("Waaar")
print(res) 


In [11]:
find_books_json = {
    "name": "find_books",
    "description": "Always use this tool to find books that match the query",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "query to search the books"
            },
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

In [12]:
get_book_cover_image_json = {
    "name": "get_book_cover_image",
    "description": "Always use this tool to find book's cover image that match the isbn",
    "parameters": {
        "type": "object",
        "properties": {
            "isbn": {
                "type": "string",
                "description": "isbn of the book"
            },
        },
        "required": ["isbn"],
        "additionalProperties": False
    }
}

In [13]:
record_unknown_question_about_book_json = {
    "name": "record_unknown_question_about_book",
    "description": "Always use this tool to record any question about book that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [14]:
tools = [{"type": "function", "function": find_books_json},
        {"type": "function", "function": record_unknown_question_about_book_json},
        {"type": "function", "function": get_book_cover_image_json}]


In [15]:
tools 

[{'type': 'function',
  'function': {'name': 'find_books',
   'description': 'Always use this tool to find books that match the query',
   'parameters': {'type': 'object',
    'properties': {'query': {'type': 'string',
      'description': 'query to search the books'}},
    'required': ['query'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question_about_book',
   'description': "Always use this tool to record any question about book that couldn't be answered as you didn't know the answer",
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': "The question that couldn't be answered"}},
    'required': ['question'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'get_book_cover_image',
   'description': "Always use this tool to find book's cover image that match the isbn",
   'parameters': {'type': 'object',
    'properties': {'isbn': {'type': 

In [17]:
# This is a more elegant way that avoids the IF statement.

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [ ]:
system_prompt = (
    "You are a specialist in painting. You answer questions on famous painters, "
    "their career, background, skills and work. "
    "If you are asked about a book or writer, you can use the find_books tool to get a list of books that match the query. "
    "If you are asked about the cover image of a book, you can use the get_book_cover_image tool to get the image. "
    "If you don't know the answer about a book or painter, use the record_unknown_question_about_book tool to record the question you couldn't answer. "
    "If the user is engaging in discussion, steer them towards getting in touch via email; ask for their email and record it using the record_user_details tool. "
    "With this context, please chat with the user, always staying in character as a specialist in painting."
)
